In [1]:
!pip install transformers[flax] huggingface_hub -i https://pypi.tuna.tsinghua.edu.cn/simple
import os

os.environ["HF_ENDPOINT"] = "https://hf-mirror.com"

alice's Output:

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple


bob's Output:

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple


In [2]:
from transformers import AutoTokenizer, FlaxGPT2LMHeadModel, GPT2Config
import jax.numpy as jnp

tokenizer = AutoTokenizer.from_pretrained("gpt2")
pretrained_model = FlaxGPT2LMHeadModel.from_pretrained("gpt2")

# text generate function
def text_generation(input_ids, params):
    config = GPT2Config()
    model = FlaxGPT2LMHeadModel(config=config)

    for _ in range(10):
        outputs = model(input_ids=input_ids, params=params)
        next_token_logits = outputs[0][0, -1, :]
        next_token = jnp.argmax(next_token_logits)
        input_ids = jnp.concatenate([input_ids, jnp.array([[next_token]])], axis=1)
    return input_ids

bob's Output:

No GPU/TPU found, falling back to CPU. (Set TF_CPP_MIN_LOG_LEVEL=0 and rerun for more info.)


alice's Output:

flax_model.msgpack:  32%|###1      | 157M/498M [00:00<?, ?B/s]

No GPU/TPU found, falling back to CPU. (Set TF_CPP_MIN_LOG_LEVEL=0 and rerun for more info.)


generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [3]:
# generate text on cpu

inputs_ids = tokenizer.encode('I enjoy walking with my cute dog', return_tensors='jax')
outputs_ids = text_generation(inputs_ids, pretrained_model.params)

print('-' * 65 + '\nRun on CPU:\n' + '-' * 65)
print(tokenizer.decode(outputs_ids[0], skip_special_tokens=True))
print('-' * 65)

bob's Output:

-----------------------------------------------------------------
Run on CPU:
-----------------------------------------------------------------
I enjoy walking with my cute dog, but I'm not sure if I'll ever
-----------------------------------------------------------------


alice's Output:

-----------------------------------------------------------------
Run on CPU:
-----------------------------------------------------------------
I enjoy walking with my cute dog, but I'm not sure if I'll ever
-----------------------------------------------------------------


In [4]:
import secretflow as sf

# In case you have a running secretflow runtime already.
sf.shutdown()

sf.init(['alice', 'bob', 'carol'], address='local')

alice, bob = sf.PYU('alice'), sf.PYU('bob')
conf = sf.utils.testing.cluster_def(['alice', 'bob', 'carol'])
conf['runtime_config']['fxp_exp_mode'] = 1
conf['runtime_config']['experimental_disable_mmul_split'] = True
spu = sf.SPU(conf)


def get_model_params():
    pretrained_model = FlaxGPT2LMHeadModel.from_pretrained("gpt2")
    return pretrained_model.params


def get_token_ids():
    tokenizer = AutoTokenizer.from_pretrained("gpt2")
    return tokenizer.encode('I enjoy walking with my cute dog', return_tensors='jax')


model_params = alice(get_model_params)()
input_token_ids = bob(get_token_ids)()

device = spu
model_params_, input_token_ids_ = model_params.to(device), input_token_ids.to(device)

output_token_ids = spu(text_generation)(input_token_ids_, model_params_)

bob's Output:

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
2024-06-22 22:34:23,395	WARN

(_run pid=5601) [2024-06-22 22:34:35.339] [info] [thread_pool.cc:30] Create a fixed thread pool with size 19


OutOfMemoryError: Task was killed due to the node running low on memory.
Memory on the node (IP: 172.22.0.3, ID: 034d9221bc739c9e3a30552df25de69e7ff769bd027c08fb89314bb0) where the task (task ID: 03b9e560f97723d951344fd3b1c3cb4e1c5904d101000000, name=_run, pid=5601, memory used=2.68GB) was running was 14.85GB / 15.48GB (0.958721), which exceeds the memory usage threshold of 0.95. Ray killed this worker (ID: aea6ad239630efca5481902ecd302f7089a72805735e2e6383906642) because it was the most recently scheduled task; to see more information about memory usage on this node, use `ray logs raylet.out -ip 172.22.0.3`. To see the logs of the worker, use `ray logs worker-aea6ad239630efca5481902ecd302f7089a72805735e2e6383906642*out -ip 172.22.0.3. Top 10 memory users:
PID	MEM(GB)	COMMAND
5601	2.68	ray::_run
5175	1.39	/usr/local/bin/python -m ipykernel_launcher -f /root/.local/share/jupyter/runtime/kernel-826f6140-13...
5361	0.30	/usr/local/lib/python3.10/site-packages/ray/core/src/ray/gcs/gcs_serv

alice's Output:

ModuleNotFoundError: No module named 'jax.extend.linear_util'

In [5]:
sf.__version__

bob's Output:

'1.5.0b0'

alice's Output:

NameError: name 'sf' is not defined

In [ ]:
outputs_ids = sf.reveal(output_token_ids)
print('-' * 65 + '\nRun on SPU:\n' + '-' * 65)
print(tokenizer.decode(outputs_ids[0], skip_special_tokens=True))
print('-' * 65)